**REG NO : 19BCS4018**

**NAME : CHARUMATHI C**

**COLLEGE : M.KUMARASAMY COLLEGE OF ENGINEERING,KARUR**

**Section 1: Environment setup and data loading**

In [1]:
!apt-get update -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,540 kB]
Hit:13 http://ppa.launchpa

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
!pip install -q findspark
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [8]:
import gdown

In [9]:
gdown.download_folder(id='1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ', quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

In [10]:
h_data = spark.read.csv("/content/chess/chess_wc_history_game_info.csv", header=True, inferSchema=True)
m_data = spark.read.csv("/content/chess/chess_wc_history_moves.csv", header=True, inferSchema=True)
eco_data = spark.read.csv("/content/chess/eco_codes.csv", header=True, inferSchema=True)

In [11]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [12]:
h_data.groupBy('winner').count().show()

+------------------+-----+
|            winner|count|
+------------------+-----+
|   Fridman, Daniel|    2|
|    Svidler, Peter|    4|
|   Alterman, Boris|    1|
|        Ivanchuk,V|   12|
|    Andersson, Ulf|    5|
|   Jussupow, Artur|    1|
|          Piket,Je|    3|
|    Van Wely, Loek|    4|
|       Onischuk,Al|    1|
|      Peng Xiaomin|    5|
|    Kacheishvili,G|    1|
|      Vladimirov,E|    4|
|          Luther,T|    3|
| Ivanchuk, Vassily|    4|
| Steinitz, William|   43|
|         Carlsen,M|    4|
|         Ivanov,AV|    1|
|Istratescu, Andrei|    2|
|        Tiviakov,S|    5|
|   Karpov, Anatoly|   47|
+------------------+-----+
only showing top 20 rows



In [13]:
h_data.groupBy('tournament_name').count().show()

+---------------+-----+
|tournament_name|count|
+---------------+-----+
| WorldChamp2004|   14|
| WorldChamp1894|   19|
| WorldChamp2013|   10|
| WorldChamp2007|   56|
|  FideChamp2000|  345|
|  FideChamp1993|   21|
|WorldChamp1910b|   11|
| WorldChamp1921|   14|
| WorldChamp1958|   23|
| WorldChamp1981|   18|
| WorldChamp1978|   32|
| WorldChamp1961|   21|
| WorldChamp1960|   21|
| WorldChamp1948|   50|
| WorldChamp1929|   25|
| WorldChamp1934|   26|
| WorldChamp1986|   24|
|   PCAChamp1995|   18|
| WorldChamp1886|   20|
| WorldChamp1951|   24|
+---------------+-----+
only showing top 20 rows



In [14]:
h_data.count()

2938

In [15]:
h_data.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- game_order: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- site: string (nullable = true)
 |-- date_played: string (nullable = true)
 |-- round: double (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: integer (nullable = true)
 |-- black_elo: integer (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: integer (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: integer (nullable = true)
 |-- winner_loser_elo_diff: integer (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- tournament_name: string (nullable = true)



**Section 2: Using pySpark for data analysis**

**1. List of Winners of Each World champions Trophy.**

In [16]:
from pyspark.sql.functions import col
h_data1 = h_data.filter(col("winner") != "draw")
h_data1=h_data1.select("winner","tournament_name")
h_data1=h_data1.dropDuplicates(["winner","tournament_name"]).show()

+------------------+---------------+
|            winner|tournament_name|
+------------------+---------------+
|          Bareev,E|  FideChamp2002|
|           Dreev,A|  FideChamp2002|
|    Andersson, Ulf|  FideChamp1999|
|    Svidler, Peter|  FideChamp1998|
|         Ehlvest,J|  FideChamp2002|
|         Delchev,A|  FideChamp2004|
|      Ponomariov,R|  FideChamp2002|
|      Zvjaginsev,V|  FideChamp2002|
| Steinitz, William| WorldChamp1890|
|          Kudrin,S|  FideChamp2004|
|Istratescu, Andrei|  FideChamp1998|
|          Sakaev,K|  FideChamp2002|
|        Ciuksyte,D|  FideChamp2002|
|         Lautier,J|  FideChamp2002|
|      Psakhis, Lev|  FideChamp1999|
|     Dreev, Alexey|  FideChamp1999|
|         Kramnik,V| WorldChamp2006|
|     Lautier, Joel|  FideChamp1999|
|           Anand,V| WorldChamp2010|
|        Nakamura,H|  FideChamp2004|
+------------------+---------------+
only showing top 20 rows



**2. List of Players with number of times they have won Tournament in descending order(Max to min).**

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.functions import desc

In [18]:
h_data2=h_data.filter("winner != 'draw'").select("winner")
h_data2.groupBy("winner").count().sort(desc("count")).show()

+--------------------+-----+
|              winner|count|
+--------------------+-----+
|     Lasker, Emanuel|   52|
|     Karpov, Anatoly|   47|
|  Botvinnik, Mikhail|   46|
|   Steinitz, William|   43|
| Alekhine, Alexander|   43|
|           Topalov,V|   32|
|      Kasparov, Gary|   31|
|             Anand,V|   30|
|    Smyslov, Vassily|   24|
|            Adams,Mi|   20|
|      Kasimdzhanov,R|   19|
|          Grischuk,A|   17|
|           Svidler,P|   16|
|      Adams, Michael|   16|
|            Shirov,A|   15|
|             Dreev,A|   15|
|            Bareev,E|   15|
|   Chigorin, Mikhail|   14|
|Khalifman, Alexander|   14|
|           Euwe, Max|   14|
+--------------------+-----+
only showing top 20 rows



**7. Most and Least rated Player.**

In [19]:
h_data.select("winner_elo").describe().show()

+-------+------------------+
|summary|        winner_elo|
+-------+------------------+
|  count|              1105|
|   mean|2658.3257918552035|
| stddev| 80.15889290248214|
|    min|              2271|
|    max|              2870|
+-------+------------------+



In [20]:
df=h_data.filter(h_data.winner_elo==2870)
print("MOST RATED PLAYED")
data1=df.select("winner","winner_elo")
data2=data1.dropDuplicates().show()

MOST RATED PLAYED
+---------------+----------+
|         winner|winner_elo|
+---------------+----------+
|Carlsen, Magnus|      2870|
+---------------+----------+



In [21]:
df1=h_data.filter(h_data.winner_elo==2271)
print("LEAST RATED PLAYED")
data1=df1.select("winner","winner_elo")
data2=data1.dropDuplicates().show()

LEAST RATED PLAYED
+----------+----------+
|    winner|winner_elo|
+----------+----------+
|Ciuksyte,D|      2271|
+----------+----------+



In [22]:
m_data.select('player').show()

+----------------+
|          player|
+----------------+
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
|Nepomniachtchi,I|
|       Carlsen,M|
+----------------+
only showing top 20 rows



In [23]:
m_data.groupBy('player').count().show()

+------------------+-----+
|            player|count|
+------------------+-----+
| Hamdouchi, Hichem|  188|
|   Fridman, Daniel|  132|
|    Svidler, Peter|  617|
|   Alterman, Boris|  297|
|  Nepomniachtchi,I|  568|
|        Ivanchuk,V| 1473|
|    Andersson, Ulf| 1050|
|   Jussupow, Artur|  140|
|          Piket,Je|  301|
|    Van Wely, Loek|  658|
|       Onischuk,Al|  288|
|      Peng Xiaomin|  862|
|      Vladimirov,E|  730|
|    Kacheishvili,G|  178|
|          Luther,T|  245|
| Ivanchuk, Vassily|  302|
|Sakaev, Konstantin|   85|
| Steinitz, William| 4989|
|         Carlsen,M|  747|
|         Ivanov,AV|  234|
+------------------+-----+
only showing top 20 rows



In [ ]:
df=h_data.filter("winner_loser_elo_diff > '0'").select("winner","winner_loser_elo_diff")
df=df.dropDuplicates()